<a href="https://colab.research.google.com/github/nitinranjansharma/EVA5B2/blob/main/s4/EVA4_Session_2_nrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1,bias=False) #input -28x28 Output-26x26 RF-3
        self.batchnorm1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1,bias=False) #input -26x26 Output-24x24 RF-5
        self.batchnorm2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) #input -16x16 Output-14x14 RF-10
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1,bias=False)
        self.batchnorm3 = nn.BatchNorm2d(16) #input -14x14 Output-12x12 RF-12
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1,bias=False) #input -12x12 Output-10x10 RF-14
        self.batchnorm4 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2) #input -10x10 Output-5x5 RF-28
        self.conv5 = nn.Conv2d(16, 32, 3,bias=False) #input -5x5 Output-3x3 RF-30
        self.batchnorm5 = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32, 10, 3,bias=False) #input -3x3 Output-1x1 RF-32
        
        

    def forward(self, x):
        x = self.pool1(F.relu(self.batchnorm2(self.conv2(self.batchnorm1(F.relu(self.conv1(x)))))))
        x = self.pool2(self.batchnorm4(F.relu(self.conv4(self.batchnorm3(F.relu(self.conv3(x)))))))
        x = F.relu(self.conv6(self.batchnorm5(F.relu(self.conv5(x)))))
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,152
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           2,304
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 16, 14, 14]           2,304
       BatchNorm2d-9           [-1, 16, 14, 14]              32
        MaxPool2d-10             [-1, 16, 7, 7]               0
           Conv2d-11             [-1, 32, 5, 5]           4,608
      BatchNorm2d-12             [-1, 32, 5, 5]              64
           Conv2d-13             [-1, 10, 3, 3]           2,880
Total params: 13,496
Trainable params: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:


torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0430, Accuracy: 9882/10000 (99%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0287, Accuracy: 9914/10000 (99%)



  0%|          | 0/1875 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0202, Accuracy: 9946/10000 (99%)



  0%|          | 0/1875 [00:00<?, ?it/s]


Test set: Average loss: 0.0191, Accuracy: 9940/10000 (99%)



loss=0.0014012942556291819 batch_id=1874: 100%|██████████| 1875/1875 [00:23<00:00, 80.58it/s]



Test set: Average loss: 0.0184, Accuracy: 9941/10000 (99%)



## Reached 99.46% Accuracy first at 5th epoch 
## Consistently the Accuracy is reaching greater than 99.28% from 4th epoch 
## Number of parameters in the model - 13,496